In [77]:

!pip3 install tensorflow opencv-python mediapipe scikit-learn matplotlib

In [78]:
import cv2 
import numpy as np 
import os # easier to acess files
from matplotlib import pyplot as plt
import time
import mediapipe as mp

In [79]:
mp_holistic = mp.solutions.holistic # holistic model - make detections
mp_drawing = mp.solutions.drawing_utils # drawing utilities - draw detections




In [80]:
# image is a photo
# model is an mp_holisitic model

# Steps to use media pipe:
# 1. grab image and convert from bgr to rgb, then unwritable (saves memory)
# 2. make our detection
# 3. convert back to writeable and bgr



def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # color conversion
    # cv2Color converts images from one colorspace to another
    image.flags.writeable = False
    results = model.process(image) # make prediction from image grame
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # color conversion
    return image, results

In [81]:
# draws landmarks on photos

#draw_landmarks is helper function that makes it easy to draw onto image
# needs image, landmarks, and what type of connections you want to use
# POSE and FACE connections will provide map for different landmarks
# NOTE: applies landmark drawings to image in place


def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION) # draw face connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS) # draw pose connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # draw left hand connections
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # draw right hand connections


In [82]:
def draw_styled_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION,
                                mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1),
                                mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)) # draw face connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(80,110,10), thickness=2, circle_radius=4),
                                mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)) # draw face connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4),
                                mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)) # draw left hand connections
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                                mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2))

In [83]:
# code to access openCV


cap = cv2.VideoCapture(0) # access video cam on device port 0

# with is used to handle resource management
# set mediapipe model

# min detection is initial detection, tracking confidence is preceding tracking confidence
# if you want higher inital confidence in answer, then increase it and vice versa
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
	while cap.isOpened(): # while camera is on

		# Read feed (reading frame from webcam)
		ret, frame = cap.read()

		# make detections
		image, results = mediapipe_detection(frame, holistic)
		print(results)

		# draw landmarks
		draw_styled_landmarks(image, results)

		# show frame to screen
		cv2.imshow('OpienCV Feed', image)
		
		# break gracefully
		if cv2.waitKey(1) & 0xFF == ord('q'):
			break

# release cv2 and close all windows
cv2.destroyAllWindows()
cv2.waitKey(1)
cap.release()

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

In [84]:
# draw_landmarks(frame, results)

In [85]:

# plt.imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))

In [86]:
# # diffe
# results.face_landmarks

Extract keypoint values

In [87]:
# pose = []
# for res in results.pose_landmarks.landmark:
#     test = np.array([res.x, res.y, res.z, res.visibility])
#     pose.append(test)
# for loop way of below code

In [88]:
# pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeroes(132)
# # makes pose array with x,y,z and visibility, and makes list of lists into just a big list

# # each hand has 21 landmarks; each landmark has x, y and z coordinate values each; thus empty arrays (when hand not visible) should be 21*3=63
# lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
# rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)

# face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(1404)

In [89]:
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    # makes pose array with x,y,z and visibility, and makes list of lists into just a big list

    # each hand has 21 landmarks; each landmark has x, y and z coordinate values each; thus empty arrays (when hand not visible) should be 21*3=63
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)

    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    return np.concatenate([pose, face, lh, rh])

In [90]:
# extract_keypoints(results)

In [91]:
# path for exported data, numpy arrays
DATA_PATH = os.path.join("MP_Data")

#Actions that we try to detect
actions = np.array(['hello', 'thanks', 'iloveyou', "yes"])

# thirty videos with of data
no_sequences = 30

# videos are going to be 30 frames of length
sequence_length = 30

#1662 datapoints, 30 

In [92]:
# for each action we have a folder
# hello
# - each action has no_sequence folders 
# - 0 (frame 1, frame 2, ... Frame 30)
# - 1
# - 2
# ... 
# - 29
# - - each numbered folder will have 30 frames (30 arrays) (sequence lenght)


# thanks
# i love you


for action in actions:
    # for each action
    for sequence in range(no_sequences):
        try:
            # if folder already exist, will pass, else make numbered folders in action
            os.makedirs(os.path.join(DATA_PATH, action, str(sequence)))
        except:
            pass

In [93]:
# # code to access openCV


# cap = cv2.VideoCapture(0) # access video cam on device port 0

# # with is used to handle resource management
# # set mediapipe model

# # min detection is initial detection, tracking confidence is preceding tracking confidence
# # if you want higher inital confidence in answer, then increase it and vice versa
# with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
	
#     # replacing while cap is open
#     for action in actions:
#           if action != "yes":
#                  continue
#           for sequence in range(no_sequences):
#                 for frame_num in range(sequence_length):
#                             # Read feed (reading frame from webcam)
#                             ret, frame = cap.read()

#                             # make detections
#                             image, results = mediapipe_detection(frame, holistic)
#                             print(results)

#                             # draw landmarks
#                             draw_styled_landmarks(image, results)

                            
                            
#                             # apply weight logic
#                             if frame_num == 0:
#                                   # image, text, position, font, font size, color
#                                   cv2.putText(image, 'Start collection', (120, 200), 
#                                   cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255,0), 5, cv2.LINE_AA)
#                                   cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15, 50), 
#                                   cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255,0), 5, cv2.LINE_AA)
#                                   cv2.waitKey(2000) # 2 second break
#                             else:
#                                   cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15, 50), 
#                                   cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255,0), 5, cv2.LINE_AA)

#                             # show frame to screen
#                             cv2.imshow('OpenCV Feed', image)

#                             # new -- export keypoints
#                             keypoints = extract_keypoints(results)
#                             npy_path = os.path.join(DATA_PATH, action, str(sequence), str(frame_num))
#                             np.save(npy_path, keypoints)
                            
                            
#                             # gracefully break
#                             if cv2.waitKey(1) & 0xFF == ord('q'):
#                                   break

# # release cv2 and close all windows
# cv2.destroyAllWindows()
# cv2.waitKey(1)
# cap.release()

In [94]:
# cap.release()
# cv2.destroyAllWindows()

In [95]:
from sklearn.model_selection import train_test_split # used for training and testing
from tensorflow.keras.utils import to_categorical # used to make labels

In [96]:
label_map = {label:num for num, label in enumerate(actions)}
label_map

{'hello': 0, 'thanks': 1, 'iloveyou': 2, 'yes': 3}

In [97]:
sequences, labels = [], []
# sequences are feature data (X), labels are target data (y)
for action in actions:
    for sequence in range(no_sequences):
        window = [] # all frames for specific sequence (video)
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            # loads numpy frame 0, frame 1, .. 
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])
        # append 

In [98]:
X = np.array(sequences) # makes to np array
y = to_categorical(labels).astype(int) # uses one hot encoding to prevent bias


In [99]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05) # splits data

In [100]:
from tensorflow.keras.models import Sequential # Sequential lets you build a sequential NN
from tensorflow.keras.layers import LSTM, Dense # LSTM is temporal (involves time) and lets build model
from tensorflow.keras.callbacks import TensorBoard # allows to logging in tensor board

In [101]:
# Tensorboard is webapp to see neural network training

log_dir = os.path.join("Logs")
tb_callback = TensorBoard(log_dir=log_dir)

In [102]:
model = Sequential() # easy to make neural network

# 3 sets of lstm levels 
# first adds 64 neurons, and if it needs to pass to next level return seqs must be true, 30 frames with 1662 datapoints
# lstm layer is good for long term sequences
# dont return lstm layer on last layer
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30, 1662)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

# actions will turn output into 3 layers, and softmax will show probabiliies from sum

In [103]:
model.compile(optimizer='Adam', loss="categorical_crossentropy", metrics=["categorical_accuracy"])
# loss function must be categorical (when multiclass data), binary class uses binary entropy, regression is mean square error

In [104]:
# model.fit(X_train, y_train, epochs=100, callbacks=[tb_callback])

Epoch 1/100
4/4 [==============================] - 6s 56ms/step - loss: 2.3002 - categorical_accuracy: 0.1842
Epoch 2/100
4/4 [==============================] - 0s 52ms/step - loss: 1.3990 - categorical_accuracy: 0.2632
Epoch 3/100
4/4 [==============================] - 0s 50ms/step - loss: 2.4187 - categorical_accuracy: 0.2193
Epoch 4/100
4/4 [==============================] - 0s 53ms/step - loss: 1.5890 - categorical_accuracy: 0.2368
Epoch 5/100
4/4 [==============================] - 0s 59ms/step - loss: 1.4099 - categorical_accuracy: 0.3246
Epoch 6/100
4/4 [==============================] - 0s 51ms/step - loss: 1.3883 - categorical_accuracy: 0.2807
Epoch 7/100
4/4 [==============================] - 0s 50ms/step - loss: 1.4069 - categorical_accuracy: 0.2807
Epoch 8/100
4/4 [==============================] - 0s 58ms/step - loss: 1.3813 - categorical_accuracy: 0.3421
Epoch 9/100
4/4 [==============================] - 0s 69ms/step - loss: 1.3608 - categorical_accuracy: 0.3772
Epoch 10/1

In [105]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_6 (LSTM)               (None, 30, 64)            442112    
                                                                 
 lstm_7 (LSTM)               (None, 30, 128)           98816     
                                                                 
 lstm_8 (LSTM)               (None, 64)                49408     
                                                                 
 dense_6 (Dense)             (None, 64)                4160      
                                                                 
 dense_7 (Dense)             (None, 32)                2080      
                                                                 
 dense_8 (Dense)             (None, 4)                 132       
                                                                 
Total params: 596708 (2.28 MB)
Trainable params: 59670

In [106]:
res = model.predict(X_test)
actions[np.argmax(res[3])]

1/1 [==============================] - 1s 623ms/step


'iloveyou'

In [107]:
actions[np.argmax(y_test[3])]

'iloveyou'

In [108]:
# model.save('gen5.h5') # saves weights
model.load_weights('gen5.h5') # loads weights

In [109]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [110]:
yhat = model.predict(X_test)

1/1 [==============================] - 0s 46ms/step


In [111]:
ytrue = np.argmax(y_test, axis=1).tolist() # conv
yhat = np.argmax(yhat, axis =1).tolist()

In [112]:
multilabel_confusion_matrix(ytrue, yhat)

array([[[4, 0],
        [0, 2]],

       [[4, 0],
        [0, 2]],

       [[4, 1],
        [0, 1]],

       [[5, 0],
        [1, 0]]], dtype=int64)

In [113]:
accuracy_score(ytrue, yhat)


0.8333333333333334

In [114]:
# needs 30 frames to make a movement
sequence = []
sentence = []
threshold = 0.4

# code to access openCV

# cap = cv2.VideoCapture("../uploads/video.mp4")  # Access video file # access video cam on device port 0
cap = cv2.VideoCapture(0) # access video cam on device port 0

# with is used to handle resource management
# set mediapipe model

# min detection is initial detection, tracking confidence is preceding tracking confidence
# if you want higher inital confidence in answer, then increase it and vice versa
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
	while cap.isOpened(): # while camera is on

		# Read feed (reading frame from webcam)
		ret, frame = cap.read()

		# make detections
		image, results = mediapipe_detection(frame, holistic)
		# print(results)

		# draw landmarks
		draw_styled_landmarks(image, results)
		
        # 2. prediction logic
		keypoints = extract_keypoints(results)
		sequence.append(keypoints)
		sequence = sequence[-30:]
		
		if len(sequence) == 30:
			res = model.predict(np.expand_dims(sequence, axis=0))[0]
			print(actions[np.argmax(res)])

		# show frame to screen
		cv2.imshow('OpenCV Feed', image)
		
		# break gracefully
		if cv2.waitKey(1) & 0xFF == ord('q'):
			break

# release cv2 and close all windows
cv2.destroyAllWindows()
cap.release()

1/1 [==============================] - 0s 36ms/step
iloveyou
1/1 [==============================] - 0s 30ms/step
iloveyou
1/1 [==============================] - 0s 24ms/step
iloveyou
1/1 [==============================] - 0s 24ms/step
iloveyou
1/1 [==============================] - 0s 30ms/step
iloveyou
1/1 [==============================] - 0s 14ms/step
iloveyou
1/1 [==============================] - 0s 28ms/step
iloveyou
1/1 [==============================] - 0s 29ms/step
iloveyou
1/1 [==============================] - 0s 24ms/step
iloveyou
1/1 [==============================] - 0s 42ms/step
iloveyou
1/1 [==============================] - 0s 16ms/step
iloveyou
1/1 [==============================] - 0s 17ms/step
iloveyou
1/1 [==============================] - 0s 34ms/step
iloveyou
1/1 [==============================] - 0s 30ms/step
iloveyou
1/1 [==============================] - 0s 17ms/step
iloveyou
1/1 [==============================] - 0s 33ms/step
iloveyou
1/1 [===================